In [1]:
# copy from main_draft-1sted | intended to copy all maniplution of dataset and 
# relevant vairables in order to reduse clus when implimented in finale draft

import warnings # Got an irritating warning
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows", None)
pd.set_option("float_format", "{:f}".format)

dataset = pd.read_csv(r"C:\Users\Sverre\Documents\VS Studio. filer\INFO284\Obli\Info284_Project\Exam Task\Dataset\elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep = ";")

# Dataset where the species isn"t the same as the main-species
bycatch = dataset[dataset["Art FAO"] != dataset["Hovedart FAO"]]

# Dataset where the species is the same as the main-species
main_species = dataset[dataset["Art FAO"] == dataset["Hovedart FAO"]]

# Excluding all features that aren't relevant
dataset = dataset[["Hovedart FAO", "Art FAO", "Lengdegruppe", "Redskap FAO", "Rundvekt", "Hovedområde start"]]

# Drop all rows where Hovedart FAO and Art FAO is NaN
dataset = dataset.dropna(subset=["Hovedart FAO", "Art FAO"])

# Make new category in lengdegruppe
dataset["Lengdegruppe"] = dataset["Lengdegruppe"].fillna("Stortare båter")

# Drop Na values
dataset = dataset.dropna(subset=["Redskap FAO"])

# Dropping Na values
dataset = dataset.dropna(subset=["Hovedområde start"])

# Create a new binary feature to use as target variable. 
dataset["Is_Bycatch"] = (dataset["Hovedart FAO"] != dataset["Art FAO"])

# ==========================================================================|

# Changing categorical data into numeric data using "cat.codes"
dataset["Art FAO"] = dataset["Art FAO"].astype("category")
dataset["Art FAO Codes"] = dataset["Art FAO"].cat.codes

dataset["Lengdegruppe"] = dataset["Lengdegruppe"].astype("category")
dataset["Lengdegruppe Codes"] = dataset["Lengdegruppe"].cat.codes

dataset["Hovedområde start"] = dataset["Hovedområde start"].astype("category")
dataset["Hovedområde start Codes"] = dataset["Hovedområde start"].cat.codes 

dataset["Redskap FAO"] = dataset["Redskap FAO"].astype("category")
dataset["Redskap FAO Codes"] = dataset["Redskap FAO"].cat.codes 

# Excluding the old features
dataset = dataset[["Art FAO Codes", "Lengdegruppe Codes", "Redskap FAO Codes", 
                   "Rundvekt", "Hovedområde start Codes", "Is_Bycatch"]]


from sklearn.model_selection import train_test_split

# Splitting dataset 
X = dataset[["Art FAO Codes", "Lengdegruppe Codes", "Redskap FAO Codes", "Rundvekt", "Hovedområde start Codes"]]
y = dataset["Is_Bycatch"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer

# Scaling only "Rundvekt" and not the others
ct = ColumnTransformer(
    [("scale", RobustScaler(), ["Rundvekt"])],
    remainder="passthrough"
)

# Fitting the scaler to the training set and not the test set to prevent data leakage
X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.transform(X_test)


from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import precision_recall_curve
from sklearn.datasets import make_blobs
from sklearn.pipeline import Pipeline

## Our choices
- Neural network

"Neural networks have a considerable succsess in low-level reasoning for with there is aboundant training data ... one reason is that they are very flexible and can invent features ... as far as learning is conserned, neural network provide a different measure of simplicity as a learning bias then, for eksample, decision trees. Multilayer neural networks, like decision tree, can represent any function of a set of discreat features."

### Neural Network

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


In [4]:

mlp = MLPClassifier(verbose=False, hidden_layer_sizes=(10, 5), learning_rate_init=0.001, random_state=50)

mlp.fit(X_train_scaled, y_train)

y_test_pred = mlp.predict(X_train_scaled)
train_accuracy = accuracy_score(y_test_pred, y_train)
print("Training accuracy: ", train_accuracy)


y_pred = mlp.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")


Training accuracy:  0.8829404965888189
Accuracy: 0.8815974096060443
Confusion Matrix: [[17925  5362]
 [ 3414 47419]]
0.8829404965888189 0.8815974096060443


The low diffeence between the accurasy and training accuracy indicateds that the model is genralsierign well to unseen data and shows little to no signs of overfitting. The low number of false posetiv and false negative indicates that the model is able to 

In [4]:

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import precision_recall_curve
from sklearn.datasets import make_blobs
from sklearn.pipeline import Pipeline

In [6]:
skf = StratifiedKFold(n_splits=10)

training_acc_list = []
testing_acc_list =[]
con_x_list = []
o_u_fitting_list= []

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    ct = ColumnTransformer(
        [("scale", RobustScaler(), ["Rundvekt"])],
        remainder="passthrough")
    X_train_scaled = ct.fit_transform(X_train)
    X_test_scaled = ct.transform(X_test)

    # aplying split to algo
    mlp = MLPClassifier(verbose=False, hidden_layer_sizes=(10, 5), learning_rate_init=0.001, random_state=50)

    mlp.fit(X_train_scaled, y_train)

    y_test_pred = mlp.predict(X_train_scaled)
    train_accuracy = accuracy_score(y_test_pred, y_train)
    training_acc_list.append(train_accuracy)

    y_pred = mlp.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    testing_acc_list.append(accuracy)

    o_u_fitting_list.append(accuracy - train_accuracy)
    con_x_list.append(confusion_matrix(y_test, y_pred))

print(training_acc_list, "\n", 
      testing_acc_list, "\n", 
      con_x_list, "\n", 
      o_u_fitting_list)

    

[0.8829404965888189, 0.8829404965888189, 0.8829404965888189, 0.8829404965888189, 0.8829404965888189, 0.8829404965888189, 0.8829404965888189, 0.8829404965888189, 0.8829404965888189, 0.8829404965888189] 
 [0.8815974096060443, 0.8815974096060443, 0.8815974096060443, 0.8815974096060443, 0.8815974096060443, 0.8815974096060443, 0.8815974096060443, 0.8815974096060443, 0.8815974096060443, 0.8815974096060443] 
 [array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]], dtype=int64), array([[17925,  5362],
       [ 3414, 47419]

There seems that there is little to no sign of biases in the dataset proven by the fackt that the training and testing accursy is indentical across all folds. It is also indendical to the basic implimentation and we then know that the shuffel of the train/test split is sussisent to ensure that there is no large ddifferenses bwtween the training and testing setts, therefor cross validation is not needed in the following grid seartch. 

In [ ]:
parameters = {"learning_rate_init":[0.01, 0.001, 0.005],
              "activation": ["identity", "logistic", "tanh", "relu"],
              "learning_rate": ["constant", "invscaling", "adaptive"],
              "hidden_layer_sizes": [(10, 5), (4, 2), (4, 4), (2, 2)]            
} 

mlp = MLPClassifier(verbose=True, 
                    hidden_layer_sizes=(10, 5), 
                    random_state=50)

gs = GridSearchCV(estimator=mlp,
                  param_grid=parameters,
                  n_jobs=-1,
                  cv=2,
                  verbose=2)

gs.fit(X_train_scaled, y_train)
best_model = gs.best_estimator_
accuracy_best_model = best_model.score(X_test_scaled, y_test)

In [13]:
print(accuracy_best_model)

0.3371424716675661


Adam: - stoc grad
batch_size
learning_rate_init
#max_iter
#shuffle
beta_1
beta_2
epsilon
n_iter_no_change


sgd: - stoc grad
batch_size
learning_rate
learning_rate_init
power_t
#max_iter
#shuffle
momentum
n_iter_no_change


lbfgs:


